In [42]:
'''
设置全局的变量
导入包
'''
import tushare as ts
import datetime
from dateutil.relativedelta import relativedelta
from docx import Document

# 全局的业务变量
class BusinessVal:
    # 标的资产，暂时只支持标的资产是一种指数
    underlyingAsset = '000001.SH'
    # 敲出比例
    knockOutRatio = 1.00
    # 敲入比例
    knockInRatio = 0.80
    # 期限，单位是月
    timeLimit = 12
    # start_date
    startDate = '20100101'
    # end_date
    endDate='20200522'
    # 月利率
    monthlyRate = 0.1
    # 保证金
    deposit = 20
    # 名义本金倍数，名义本金除以保证金
    multiple = 5

# 全局的技术变量
class TechVal:
    # tushare_token
    tushareToken = '19280d5f247ae49caae39e24a4363308c5396514275535ed770716c2'

# 初始化 tushare
ts.set_token(TechVal.tushareToken)
pro = ts.pro_api()

print('标的资产   ' + BusinessVal.underlyingAsset)
print('敲出比例   ' + str(BusinessVal.knockOutRatio))
print('敲入比例   ' + str(BusinessVal.knockInRatio))
print('期限   ' + str(BusinessVal.timeLimit))
print('回测起始时间   ' + BusinessVal.startDate)
print('回测终止时间   ' + BusinessVal.endDate)


标的资产   000001.SH
敲出比例   1.0
敲入比例   0.8
期限   12
回测起始时间   20100101
回测终止时间   20200522


In [48]:
'''
获取行情数据
'''
quote = pro.index_daily(ts_code=BusinessVal.underlyingAsset, start_date = BusinessVal.startDate, end_date=BusinessVal.endDate)
# 行反转，这样第一行是最早的日期
quote = quote.reindex(index=quote.index[::-1])
print(quote)
quote.to_excel(BusinessVal.underlyingAsset + ' ' + '文档生成于' + datetime.datetime.now().strftime('%y%m%d%H%M%S') + '.xlsx')

        ts_code trade_date      close       open       high        low  \
2522  000001.SH   20100104  3243.7600  3289.7500  3295.2790  3243.3190   
2521  000001.SH   20100105  3282.1790  3254.4680  3290.5120  3221.4620   
2520  000001.SH   20100106  3254.2150  3277.5170  3295.8680  3253.0440   
2519  000001.SH   20100107  3192.7760  3253.9910  3268.8190  3176.7070   
2518  000001.SH   20100108  3195.9970  3177.2590  3198.9200  3149.0170   
...         ...        ...        ...        ...        ...        ...   
4     000001.SH   20200518  2875.4176  2872.5244  2889.9795  2862.2709   
3     000001.SH   20200519  2898.5760  2897.6867  2900.2187  2887.5768   
2     000001.SH   20200520  2883.7378  2896.4657  2896.4657  2876.1766   
1     000001.SH   20200521  2867.9237  2890.7213  2891.5177  2864.2073   
0     000001.SH   20200522  2813.7654  2863.0462  2863.0462  2808.0177   

      pre_close   change  pct_chg          vol        amount  
2522  3277.1390 -33.3790  -1.0185  109447927.0  

In [44]:
'''
定义一个函数，获取最近的一个交易日
'''
# 交易日历
tradeCalendar = pro.trade_cal(is_open='1')
print(tradeCalendar)

def getLeastTradeDate(searchData):
    '''
    获取searchData的在未来最近一个交易日
    :param searchData: 传入日期
    :return: outDate
    '''
    
    # 如果结果是空，说明不是交易日
    while (tradeCalendar.loc[tradeCalendar['cal_date'] == searchData]).empty:
        searchData = (datetime.datetime.strptime(searchData,'%Y%m%d') + relativedelta(days=1)).strftime('%Y%m%d')
    
    rntDate = searchData
    return rntDate

def isTradeDate(checkDate):
    '''
    判断checkData是否是交易日
    :param checkDate: 
    :return: 
    '''
    # 如果结果是空，说明不是交易日
    if (tradeCalendar.loc[tradeCalendar['cal_date'] == checkDate]).empty:
        return False
    else:
        return True
    

     exchange  cal_date  is_open
0         SSE  19901219        1
1         SSE  19901220        1
2         SSE  19901221        1
3         SSE  19901224        1
4         SSE  19901225        1
...       ...       ...      ...
7338      SSE  20201225        1
7339      SSE  20201228        1
7340      SSE  20201229        1
7341      SSE  20201230        1
7342      SSE  20201231        1

[7343 rows x 3 columns]


In [45]:
# 模拟
def simulate(anchorDate,anchorPrice,knockOutPrice,knockInPrice,observeList):
    '''
    模拟
    :param anchorDate: 锚定日期
    :param anchorPrice: 锚定价格
    :param knockOutPrice: 敲出价格
    :param knockInPrice: 敲入价格
    :param observeList: 观察日列表
    :return: 
    '''
    # 是否曾经敲入过
    isKnockIn = False
    # 是否曾经敲出过
    isKnockOut = False
    # 模拟日期
    simulateDate = anchorDate
    # 到期日
    endDate = observeList[-1]
    # 如果没到期
    while datetime.datetime.strptime(simulateDate,'%Y%m%d') <= datetime.datetime.strptime(endDate,'%Y%m%d'):
        # 如果不是交易日
        if not isTradeDate(checkDate=simulateDate):
            simulateDate = (datetime.datetime.strptime(simulateDate,'%Y%m%d') + relativedelta(days=1)).strftime('%Y%m%d')
            continue
        # 如果是交易日
        # 获取行情数据
        simulateQuote = quote.loc[quote['trade_date'] == simulateDate]
        # 判断是否敲入
        if simulateQuote['close'].values[0] < knockInPrice:
           isKnockIn = True
        # 判断是否是观察日
        observeIndex = 0
        if simulateDate in observeList:
            observeIndex = observeIndex + 1
            # 判断是否敲出
            if simulateQuote['close'].values[0] > knockOutPrice:
                isKnockOut = True
            # 没有敲入，没有敲出
            if isKnockIn == False & (isKnockOut == False & (simulateDate == endDate)):
                payType = '没有敲入，没有敲出'
                payDate = simulateDate
                payOff = BusinessVal.deposit + observeIndex * BusinessVal.deposit * BusinessVal.monthlyRate * BusinessVal.multiple
                # return (payType,payDate,payOff)
                break
            # 没有敲入，有敲出
            if isKnockIn == False & isKnockOut == True:
                payType = '没有敲入，有敲出'
                payDate = simulateDate
                payOff = BusinessVal.deposit + observeIndex * BusinessVal.deposit * BusinessVal.monthlyRate * BusinessVal.multiple\
                # return (payType,payDate,payOff)
                break
            # 有敲入，有敲出
            if isKnockIn == True & isKnockOut == True:
                payType = '有敲入，有敲出'
                payDate = simulateDate
                payOff = BusinessVal.deposit + observeIndex * BusinessVal.deposit * BusinessVal.monthlyRate * BusinessVal.multiple\
                # return (payType,payDate,payOff)
                break
            # 有敲入，没有敲出
            if isKnockIn == True & (isKnockOut == False & (simulateDate == endDate)):
                payType = '有敲入，有敲出'
                payDate = simulateDate
                payOff = BusinessVal.deposit * (1 - (simulateQuote['close'].values[0] - anchorPrice))
                # return (payType,payDate,payOff)
                break
            
        simulateDate = (datetime.datetime.strptime(simulateDate,'%Y%m%d') + relativedelta(days=1)).strftime('%Y%m%d')
    
    return (payType,payDate,payOff)
        
    

In [46]:

# 实例化一个document
doc = Document()
doc.add_heading('基础信息',level=0)
doc.add_paragraph('标的资产' + ' ' * 3 + str(BusinessVal.underlyingAsset))
doc.add_paragraph('敲出比例' + ' ' * 3 + str(BusinessVal.knockOutRatio))
doc.add_paragraph('敲入比例' + ' ' * 3 + str(BusinessVal.knockInRatio))
doc.add_paragraph('期限(月)' + ' ' * 3 + str(BusinessVal.timeLimit))
doc.add_paragraph('历史行情起始日期' + ' ' * 3 + str(BusinessVal.startDate))
doc.add_paragraph('历史行情结束日期' + ' ' * 3 + str(BusinessVal.endDate))
doc.add_paragraph('月利率' + ' ' * 3 + str(BusinessVal.monthlyRate))
doc.add_paragraph('保证金' + ' ' * 3 + str(BusinessVal.deposit))
doc.add_paragraph('杠杆' + ' ' * 3 + str(BusinessVal.multiple))

# 遍历所有行情
for index, row in quote.iterrows():
    # 锚定日期
    anchorDate = row['trade_date']
    print(anchorDate)
    # 锚定价格
    anchorPrice = row['close']
    # 敲出价格
    knockOutPrice = anchorPrice * BusinessVal.knockOutRatio
    # 敲入价格
    knockInPrice = anchorPrice * BusinessVal.knockInRatio
    # 观察日列表
    observeList = []
    for i in range(1,BusinessVal.timeLimit + 1,1):
        observeList.append(getLeastTradeDate((datetime.datetime.strptime(anchorDate,'%Y%m%d') + relativedelta(months=i)).strftime('%Y%m%d')))
    # 最后一个行情日
    lastQuoteDate = quote.iloc[[-1]]['trade_date'].values[0]
    # 添加标题
    doc.add_heading(anchorDate + ' ' * 3 + str(anchorPrice) ,level=0)
    doc.add_paragraph('锚定日期' + ' ' * 3 + str(anchorDate))
    doc.add_paragraph('锚定价格' + ' ' * 3 + str(anchorPrice))
    doc.add_paragraph('敲出价格' + ' ' * 3 + str(knockOutPrice))
    doc.add_paragraph('敲入价格' + ' ' * 3 + str(knockInPrice))
    doc.add_paragraph('敲出观察日列表' + ' ' * 3 + ','.join(observeList))
    doc.add_paragraph('最后一个行情日' + ' ' * 3 + str(lastQuoteDate))
    if datetime.datetime.strptime(observeList[-1],'%Y%m%d') > datetime.datetime.strptime(lastQuoteDate,'%Y%m%d'):
        doc.add_paragraph('最后一个敲出观察日大于最后一个行情日，无法进行模拟')
        continue
    payType,payDate,payOff = simulate(anchorDate,anchorPrice,knockOutPrice,knockInPrice,observeList)
    doc.add_paragraph('结束类型' + ' ' * 3 + str(payType))
    doc.add_paragraph('结束日期' + ' ' * 3 + str(payDate))
    doc.add_paragraph('收益' + ' ' * 3 + str(payOff))

fileName = BusinessVal.underlyingAsset + ' ' + '文档生成于' + datetime.datetime.now().strftime('%y%m%d%H%M%S') + '.docx'
doc.save(fileName)
print(fileName)
print('FINISH')

20100104
20100105
20100106
20100107
20100108
20100111
20100112
20100113
20100114
20100115
20100118
20100119
20100120
20100121
20100122
20100125
20100126
20100127
20100128
20100129
20100201
20100202
20100203
20100204
20100205
20100208
20100209
20100210
20100211
20100212
20100222
20100223
20100224
20100225
20100226
20100301
20100302
20100303
20100304
20100305
20100308
20100309
20100310
20100311
20100312
20100315
20100316
20100317
20100318
20100319
20100322
20100323
20100324
20100325
20100326
20100329
20100330
20100331
20100401
20100402
20100406
20100407
20100408
20100409
20100412
20100413
20100414
20100415
20100416
20100419
20100420
20100421
20100422
20100423
20100426
20100427
20100428
20100429
20100430
20100504
20100505
20100506
20100507
20100510
20100511
20100512
20100513
20100514
20100517
20100518
20100519
20100520
20100521
20100524
20100525
20100526
20100527
20100528
20100531
20100601
20100602
20100603
20100604
20100607
20100608
20100609
20100610
20100611
20100617
20100618
20100621
2

KeyboardInterrupt: 